In [1]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies.b_testing_strats import RSICross, SmaCross
from src.utils import load_data

warnings.filterwarnings("ignore")
pd.set_option("display.float_format", "{:,.2f}".format)

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
ticker = "XLF"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [3]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1998-12-22 00:00:00-05:00,11.49,11.49,11.37,11.43,55887,0.00,0.00,0.00
1998-12-23 00:00:00-05:00,11.38,11.59,11.38,11.59,78784,0.00,0.00,0.00
1998-12-24 00:00:00-05:00,11.59,11.67,11.56,11.67,43824,0.00,0.00,0.00
1998-12-28 00:00:00-05:00,11.66,11.66,11.49,11.52,51948,0.00,0.00,0.00
1998-12-29 00:00:00-05:00,11.53,11.64,11.39,11.64,100819,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
2024-12-02 00:00:00-05:00,51.42,51.49,50.70,50.83,55483600,0.00,0.00,0.00
2024-12-03 00:00:00-05:00,51.13,51.13,50.45,50.45,46924300,0.00,0.00,0.00
2024-12-04 00:00:00-05:00,50.40,50.44,50.09,50.26,41017400,0.00,0.00,0.00


In [4]:
strategy = SmaCross
strategy.n1 = 50
strategy.n2 = 125
bt_sma = Backtest(data["2020":], strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2020-01-02 00:00...
End                       2024-12-06 00:00...
Duration                   1800 days 00:00:00
Exposure Time [%]                       66.18
Equity Final [$]                    19,876.10
Equity Peak [$]                     20,242.52
Return [%]                              98.76
Buy & Hold Return [%]                   78.14
Return (Ann.) [%]                       14.96
Volatility (Ann.) [%]                   17.44
Sharpe Ratio                             0.86
Sortino Ratio                            1.51
Calmar Ratio                             0.66
Max. Drawdown [%]                      -22.81
Avg. Drawdown [%]                       -2.89
Max. Drawdown Duration      960 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                    4
Win Rate [%]                            50.00
Best Trade [%]                          66.20
Worst Trade [%]                         -8.22
Avg. Trade [%]                    

In [5]:
bt_sma.plot()

Row(id='1450', ...)

In [6]:
%%time
stats = bt_sma.optimize(
    n1=range(20, 70, 10),
    n2=range(100, 250, 20),
    maximize="Equity Final [$]",
    constraint=lambda param: param.n1 < param.n2,
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 1.67 s, sys: 57.5 ms, total: 1.73 s
Wall time: 1.73 s


Start                     2020-01-02 00:00...
End                       2024-12-06 00:00...
Duration                   1800 days 00:00:00
Exposure Time [%]                       65.06
Equity Final [$]                    23,468.75
Equity Peak [$]                     23,901.21
Return [%]                             134.69
Buy & Hold Return [%]                   78.14
Return (Ann.) [%]                       18.90
Volatility (Ann.) [%]                   17.61
Sharpe Ratio                             1.07
Sortino Ratio                            1.98
Calmar Ratio                             1.06
Max. Drawdown [%]                      -17.75
Avg. Drawdown [%]                       -2.52
Max. Drawdown Duration      740 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                    3
Win Rate [%]                            66.67
Best Trade [%]                          58.57
Worst Trade [%]                         -1.28
Avg. Trade [%]                    

In [7]:
bt_sma.plot()

Row(id='2182', ...)

In [8]:
strategy = RSICross
strategy.low_threshold = 36
strategy.high_threshold = 80
bt_rsi = Backtest(data["2023":], strategy, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_rsi.run()
stats

Start                     2023-01-03 00:00...
End                       2024-12-06 00:00...
Duration                    703 days 00:00:00
Exposure Time [%]                       73.46
Equity Final [$]                    12,905.75
Equity Peak [$]                     13,143.83
Return [%]                              29.06
Buy & Hold Return [%]                   51.35
Return (Ann.) [%]                       14.14
Volatility (Ann.) [%]                   15.42
Sharpe Ratio                             0.92
Sortino Ratio                            1.54
Calmar Ratio                             0.90
Max. Drawdown [%]                      -15.73
Avg. Drawdown [%]                       -3.49
Max. Drawdown Duration      270 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   28
Win Rate [%]                            35.71
Best Trade [%]                          23.39
Worst Trade [%]                         -4.33
Avg. Trade [%]                    

In [9]:
bt_rsi.plot()

Row(id='2939', ...)

In [10]:
%%time
stats = bt_rsi.optimize(
    low_threshold=range(20, 41, 2),
    high_threshold=range(60, 81, 2),
    maximize="Equity Final [$]",
)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 4.02 s, sys: 79.1 ms, total: 4.1 s
Wall time: 4.08 s


Start                     2023-01-03 00:00...
End                       2024-12-06 00:00...
Duration                    703 days 00:00:00
Exposure Time [%]                       69.34
Equity Final [$]                    13,459.78
Equity Peak [$]                     13,708.09
Return [%]                              34.60
Buy & Hold Return [%]                   51.35
Return (Ann.) [%]                       16.66
Volatility (Ann.) [%]                   15.05
Sharpe Ratio                             1.11
Sortino Ratio                            1.94
Calmar Ratio                             1.06
Max. Drawdown [%]                      -15.73
Avg. Drawdown [%]                       -3.02
Max. Drawdown Duration      289 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                   28
Win Rate [%]                            35.71
Best Trade [%]                          23.39
Worst Trade [%]                         -4.33
Avg. Trade [%]                    

In [11]:
stats._strategy

<Strategy RSICross(low_threshold=36,high_threshold=78)>